# Imports

In [7]:
import pandas as pd
import numpy as np


In [124]:
import warnings
warnings.filterwarnings('ignore')

# Cleaning

In [9]:
# load the data
train = pd.read_csv('../data/train.csv')
store = pd.read_csv('../data/store.csv')

/home/mz/mambaforge/envs/ml_2023/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3552: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [287]:
train.dtypes

Date              object
Store            float64
DayOfWeek        float64
Sales            float64
Customers        float64
Open             float64
Promo            float64
StateHoliday      object
SchoolHoliday    float64
dtype: object

In [288]:
# different statistics about the data
# train.info()

# check nulls (difference between NaN and 0)
train.isnull().sum()
(train.isnull() & train.ne(0)).sum()

# see rows with NaN
train[train['Store'].isnull()]

# have look at the duplicates - they look crappy we drop them
train[train.duplicated()].head(6)

# check for question marks which are sometimes placeholder for NaN
train.select_dtypes(exclude=np.number).apply( lambda x: x.str.contains("\?", regex=True) ).any()

# what values are in StateHoliday
train.StateHoliday.unique()

train.describe().T 
train.isnull().sum()/len(train)

Date             0.000000
Store            0.030263
DayOfWeek        0.029818
Sales            0.029833
Customers        0.029934
Open             0.030083
Promo            0.030095
StateHoliday     0.030189
SchoolHoliday    0.030320
dtype: float64

# cleaning functions

In [2]:
def encode_promo_interval(df):
    """custom encoding where NaN = 0"""
    
    df_ = df.copy()
    d = {'Mar,Jun,Sept,Dec': 1, 'Feb,May,Aug,Nov':2, 'Jan,Apr,Jul,Oct':3, np.nan: 0}
    df_['PromoInterval'] = df_['PromoInterval'].map(d)

    return df_

In [3]:
import pandas as pd
import datetime

def competition_since(df):
    df_ = df.copy()

    # Fill NaN values inplaces
    df_.CompetitionOpenSinceYear.fillna(0, inplace=True)
    df_.CompetitionOpenSinceMonth.fillna(0, inplace=True)

    # Round and type convert
    df_.CompetitionOpenSinceYear = df_.CompetitionOpenSinceYear.round().astype('int')
    df_.CompetitionOpenSinceMonth = df_.CompetitionOpenSinceMonth.round().astype('int')

    today = datetime.datetime.today()
    
    # Calculate since when there is competition in month
    df_['Competition_Since_X_months'] = (today.year - df_.CompetitionOpenSinceYear) * 12 + (today.month - df_.CompetitionOpenSinceMonth)

    # competition dating from the 80' does not count
    months_since = (today.year - 1980) * 12 + today.month
    df_.loc[df_['Competition_Since_X_months'] > months_since, ['Competition_Since_X_months']] = 0

    # Set Competition_Since_X_months to 0 when there are zero months
    df_.loc[df_['Competition_Since_X_months'] <= 0, 'Competition_Since_X_months'] = 0

    # drop the columns we no longer need
    df_.drop(columns=['CompetitionOpenSinceYear', 'CompetitionOpenSinceMonth'], inplace=True)

    return df_


In [4]:
import datetime


def weeks_since_promo2(df):
    df_ = df.copy()
    
    df_.Promo2SinceWeek.fillna(0, inplace=True)
    df_.Promo2SinceYear.fillna(0, inplace=True)

    df_.Promo2SinceYear = df_.Promo2SinceYear.round().astype('int')
    df_.Promo2SinceWeek = df_.Promo2SinceWeek.round().astype('int')

    # Get the current year and week
    current_year, current_week, _ = pd.Timestamp.today().isocalendar()

    # Calculate the number of weeks since the promo start date
    df_['weeks_since_promo2'] = (current_year - df_.Promo2SinceYear) * 52 + (current_week - df_.Promo2SinceWeek)

    today = datetime.datetime.today()
    weeks_since = (today - datetime.datetime(1980, 1, 1)).days // 7
    df_.loc[df_['weeks_since_promo2'] > weeks_since, ['weeks_since_promo2']] = 0

    # drop the columns we no longer need
    df_.drop(columns=['Promo2SinceYear', 'Promo2SinceWeek'], inplace=True)

    return df_

In [5]:
def downcaster(df):
    df_ = df.copy()

    # for now 'StateHoliday', 'SchoolHoliday', are left out
    int_cols = ['DayOfWeek', 'Sales', 'Customers', 'Promo', 
                'Promo2', 'PromoInterval', 'Competition_Since_X_months', 'weeks_since_promo2']

    for col in int_cols:
        df_[col] = df_[col].astype('int')

    return df_

In [20]:
import datetime

def cleaner(df, store):
    
    df_ = df.copy()
    df_.dropna(inplace=True)

    df_ = df_[df_['Open']==1]

    df_ = df_[df_['Sales'] >=0]
    df_ = df_[df_['Sales'] !=0]

    # for now we don't account for Holidays
    df_ = df_.drop(columns=['StateHoliday', 'SchoolHoliday', 'Open'])

    # convert string Date to datetime
    df_['Date'] = pd.to_datetime(df_.Date, infer_datetime_format=True)

    # change normal days to 1 and holidays to 0
    # df_['StateHoliday'] = df_.StateHoliday.apply(lambda x: 1 if x in ['0', 0.0] else 0)

    # fill NaN with zeros for smooth imputing
    # df_['CompetitionDistance'] = df_merge['CompetitionDistance'].fillna(0).astype('int')
    
    # join cleaned train and store data 
    df_ = pd.merge(df_, store, how='outer', on='Store')

    df_ = df_.drop(columns=['Store'])
    
    df_ = encode_promo_interval(df_)

    # engineers 'Competition_Since_X_months' feature
    df_ = competition_since(df_)
    
    # engineers 'weeks_since_promo2' feature
    df_ = weeks_since_promo2(df_)

    # downcast to integers if possible
    df_ = downcaster(df_)
    
    return df_


In [21]:
df_t = cleaner(train, store)
df_t.Sales.isnull().sum()

0

In [335]:
# df_merge = cleaner(train, store)
# df_merge.head()
# df_merge.isnull().sum()
# df_merge.dtypes
# df_merge.columns

# double-checking the new df
# df_merge.StateHoliday.isnull().sum()
# df_merge.StateHoliday.unique()

# df_merge.SchoolHoliday.unique()
# df_merge.StoreType.unique()
# df_merge.Assortment.unique()
# df_merge.CompetitionOpenSinceMonth.unique()
# # df_merge.describe().T

# df_merge.dtypes
df_merge.isnull().sum()/len(df_merge)


Index(['Date', 'DayOfWeek', 'Sales', 'Customers', 'Promo', 'StoreType',
       'Assortment', 'CompetitionDistance', 'Promo2', 'PromoInterval',
       'Competition_Since_X_months', 'weeks_since_promo2'],
      dtype='object')

In [358]:
#write the cleaned data to disk
df_merge.to_csv('./data/df_clean.csv',index=False)